In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/text_matching/snli/main')

In [2]:
!pip install tensorflow-hub

In [3]:
import tensorflow as tf
import tensorflow_hub as hub

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

import numpy as np
import pprint
import logging

from pathlib import Path

TensorFlow Version 1.14.0
GPU Enabled: True


In [0]:
# stream data from text files
def gen_fn(f_path):
  label2idx = {'neutral': 0, 'entailment': 1, 'contradiction': 2,}
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      label, text1, text2 = line.split('\t')
      if label == '-':
        continue
      yield (text1.split(), text2.split()), label2idx[label]
      

def input_fn(mode, params):
  _shapes = (([None], [None]), ())
  _types = ((tf.string, tf.string), tf.int32)
  _pads = (('<pad>', '<pad>'), -1)
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    ds = tf.data.Dataset.from_generator(
      lambda: gen_fn(params['train_path'],),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.repeat()
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  if mode == tf.estimator.ModeKeys.EVAL:
    ds = tf.data.Dataset.from_generator(
      lambda: gen_fn(params['test_path'],),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  return ds

In [0]:
def elmo_repr(elmo, x, x_len):
  return elmo(inputs={'tokens': x, 'sequence_len': x_len,}, signature='tokens', as_dict=True)[params['elmo_out_node']]

In [0]:
def bi_lstm(x1, x2, seq_len_1, seq_len_2):
  lstm_fw = tf.contrib.rnn.LSTMBlockFusedCell(params['hidden_dim'], name='lstm_fused_fw')
  lstm_bw = tf.contrib.rnn.LSTMBlockFusedCell(params['hidden_dim'], name='lstm_fused_bw')
  lstm_bw = tf.contrib.rnn.TimeReversedFusedRNN(lstm_bw)
  
  t1 = tf.transpose(x1, [1,0,2])
  t2 = tf.transpose(x2, [1,0,2])
  
  o1_fw, _ = lstm_fw(t1, dtype=tf.float32, sequence_length=seq_len_1)
  o1_bw, _ = lstm_bw(t1, dtype=tf.float32, sequence_length=seq_len_1)
  
  o2_fw, _ = lstm_fw(t2, dtype=tf.float32, sequence_length=seq_len_2)
  o2_bw, _ = lstm_bw(t2, dtype=tf.float32, sequence_length=seq_len_2)
  
  t1 = tf.concat([o1_fw, o1_bw], -1)
  t2 = tf.concat([o2_fw, o2_bw], -1)
  
  x1_ = tf.transpose(t1, [1,0,2])
  x2_ = tf.transpose(t2, [1,0,2])
  return x1_, x2_


def masked_attention(x, align, mask, tile_len):
  pad = tf.fill(tf.shape(align), float('-inf'))
  mask = tf.tile(tf.expand_dims(mask, 1), [1, tile_len, 1])
  align = tf.where(tf.equal(mask, 0), pad, align)
  align = tf.nn.softmax(align)
  return tf.matmul(align, x)

  
def soft_align_attention(x1, x2, mask1, mask2):
  align12 = tf.matmul(x1, x2, transpose_b=True)
  align21 = tf.transpose(align12, [0,2,1])
  
  x1_ = masked_attention(x2, align12, mask2, tf.shape(x1)[1])
  x2_ = masked_attention(x1, align21, mask1, tf.shape(x2)[1])
  
  return x1_, x2_


def attention_pooling(x, masks, dense1, dense2, dropout, is_training):  
  # alignment
  align = dense2(dense1(x))
  align = tf.squeeze(align, -1)
  
  # masking
  paddings = tf.fill(tf.shape(align), float('-inf'))
  align = tf.where(tf.equal(masks, 0), paddings, align)

  # probability
  align = tf.nn.softmax(align)
  align = dropout(align, training=is_training)
  align = tf.expand_dims(align, -1)
 
  # weighted sum
  x = tf.squeeze(tf.matmul(x, align, transpose_a=True), -1)
  
  return x

In [0]:
def model_fn(features, labels, mode, params):
  # Flag for Dropout / Batch Norm
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  
  # Receive inputs
  raw_text1, raw_text2 = features
  batch_sz = tf.shape(raw_text1)[0]
  
  
  # Word Indexing
  vocab = tf.contrib.lookup.index_table_from_file(
        params['vocab_path'], num_oov_buckets=1)
  
  text1 = vocab.lookup(raw_text1)
  text2 = vocab.lookup(raw_text2)
  
  
  # For Masking
  seq_len1 = tf.count_nonzero(text1, 1, dtype=tf.int32)
  seq_len2 = tf.count_nonzero(text2, 1, dtype=tf.int32)
  
  mask1 = tf.sign(text1)
  mask2 = tf.sign(text2)
  
  
  # Embedding
  embedding = np.load(params['embedding_path'])
  embedding = tf.Variable(embedding, name='embedding', dtype=tf.float32)
  x1 = tf.nn.embedding_lookup(embedding, text1)
  x2 = tf.nn.embedding_lookup(embedding, text2)
  
  
  # Encoding
  elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
  e1 = elmo_repr(elmo, raw_text1, seq_len1)
  e2 = elmo_repr(elmo, raw_text2, seq_len2)
  
  dropout = tf.layers.Dropout(0.2, noise_shape=(batch_sz, 1, 1024))
  e1 = dropout(e1, training=is_training)
  e2 = dropout(e2, training=is_training)
  
  fully_connected = tf.layers.Dense(params['hidden_dim'], tf.nn.elu)
  e1 = fully_connected(e1)
  e2 = fully_connected(e2)
  
  x1 = tf.concat([x1, e1], -1)
  x2 = tf.concat([x2, e2], -1)
  
  dropout = tf.layers.Dropout(0.2, noise_shape=(batch_sz, 1, 2*params['hidden_dim']))
  x1 = dropout(x1, training=is_training)
  x2 = dropout(x2, training=is_training)
  
  x1, x2 = bi_lstm(x1, x2, seq_len1, seq_len2)
  
  
  # Interaction / Comparison
  x1_, x2_ = soft_align_attention(x1, x2, mask1, mask2)
  fn = lambda x, x_: tf.concat((x,
                                x_,
                                (x - x_),
                                (x * x_),), -1)
  x1 = fn(x1, x1_)
  x2 = fn(x2, x2_)
  
  dropout = tf.layers.Dropout(0.5)
  x1 = dropout(x1, training=is_training)
  x2 = dropout(x2, training=is_training)
  
  fully_connected = tf.layers.Dense(params['hidden_dim'], tf.nn.elu)
  x1 = fully_connected(x1)
  x2 = fully_connected(x2)
  
  
  # Encoding on top of Interaction
  dropout = tf.layers.Dropout(0.2,
                              noise_shape=(batch_sz, 1, params['hidden_dim']))
  x1 = dropout(x1, training=is_training)
  x2 = dropout(x2, training=is_training)
  
  x1, x2 = bi_lstm(x1, x2, seq_len1, seq_len2)
  
  
  # Extraction
  attn1 = tf.layers.Dense(params['hidden_dim'], tf.tanh, use_bias=False, name='attn_pool_1')
  attn2 = tf.layers.Dense(1, use_bias=False, name='attn_pool_2')
  dropout = tf.layers.Dropout(0.15)
  
  x = tf.concat((
      attention_pooling(x1, mask1, attn1, attn2, dropout, is_training),
      attention_pooling(x2, mask2, attn1, attn2, dropout, is_training),
      tf.reduce_max(x1, 1),
      tf.reduce_max(x2, 1),
  ), -1)
  
  
  # Fully Connected
  x = tf.layers.dropout(x, 0.5, training=is_training)
  
  x = tf.layers.dense(x, params['hidden_dim'], tf.nn.elu, name='fully_connected_1')
  
  x = tf.layers.dropout(x, 0.2, training=is_training)
  
  x = tf.layers.dense(x, params['hidden_dim'], tf.nn.elu, name='fully_connected_2')
  
  x = tf.layers.dropout(x, 0.2, training=is_training)
  
  logits = tf.layers.dense(x, params['num_labels'], name='output')
  
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=tf.argmax(logits, -1))
  else:
    loss_op = tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=labels, logits=logits)
    loss_op = tf.reduce_mean(loss_op)
  
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    variables = tf.trainable_variables()
    tf.logging.info('\n'+pprint.pformat(variables))
    
    grads = tf.gradients(loss_op, variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    
    global_step=tf.train.get_or_create_global_step()
    decay_lr = tf.train.exponential_decay(
        params['lr'], global_step, 100000, .25)
    
    optim = tf.train.AdamOptimizer(decay_lr)
    
    train_op = optim.apply_gradients(
      zip(grads, variables), global_step=global_step)
    
    hook = tf.train.LoggingTensorHook({'lr': decay_lr}, every_n_iter=100)
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss_op,
                                      train_op=train_op,
                                      training_hooks=[hook],)
  
  
  if mode == tf.estimator.ModeKeys.EVAL:
    acc_op = tf.metrics.accuracy(labels=labels,
                                 predictions=tf.argmax(logits, -1))
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss_op,
                                      eval_metric_ops={'acc': acc_op})

In [0]:
params = {
    'model_dir': '../model/esim',
    'log_path': '../log/esim.txt',
    'train_path': '../data/train.txt',
    'test_path': '../data/test.txt',
    'embedding_path': '../vocab/word.npy',
    'vocab_path': '../vocab/word.txt',
    'elmo_out_node': 'lstm_outputs1',
    'batch_size': 32,
    'num_samples': 550152,
    'buffer_size': 200000,
    'hidden_dim': 300,
    'lr': 4e-4,
    'clip_norm': 10.,
    'num_labels': 3,
    'num_patience': 7,
}

In [9]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
_eval_steps = params['num_samples']//params['batch_size']//10
config = tf.estimator.RunConfig(
  save_checkpoints_steps=_eval_steps,
  keep_checkpoint_max=params['num_patience']+2,)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=config,
  params=params)

# This hook early-stops model if testing accuracy is not increased
hook = tf.estimator.experimental.stop_if_no_increase_hook(
  estimator=estimator,
  metric_name='acc',
  max_steps_without_increase=params['num_patience']*_eval_steps,
  run_every_secs=None,
  run_every_steps=_eval_steps)

# Train on training data and Evaluate on testing data
train_spec = tf.estimator.TrainSpec(
  input_fn=lambda: input_fn(mode=tf.estimator.ModeKeys.TRAIN,
                            params=params),
  hooks=[hook])

eval_spec = tf.estimator.EvalSpec(
  input_fn=lambda: input_fn(mode=tf.estimator.ModeKeys.EVAL,
                            params=params),
  steps=None,
  throttle_secs=60,)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

I0704 00:36:33.273598 140182873646976 estimator.py:209] Using config: {'_model_dir': '../model/esim', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1719, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 9, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7e86d93358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
I0704 00:36:33.277031 140182873646976 estimator_training.py:186] Not using Distribute Coordinato

Reading ../data/train.txt


I0704 00:37:33.730792 140182873646976 basic_session_run_hooks.py:262] loss = 1.1356825, step = 0
I0704 00:37:33.732206 140182873646976 basic_session_run_hooks.py:262] lr = 0.0004
I0704 00:37:55.401317 140182873646976 basic_session_run_hooks.py:692] global_step/sec: 4.6143
I0704 00:37:55.407062 140182873646976 basic_session_run_hooks.py:260] loss = 0.9285493, step = 100 (21.676 sec)
I0704 00:37:55.408912 140182873646976 basic_session_run_hooks.py:260] lr = 0.00039944582 (21.677 sec)
I0704 00:38:13.858746 140182873646976 basic_session_run_hooks.py:692] global_step/sec: 5.41787
I0704 00:38:13.860819 140182873646976 basic_session_run_hooks.py:260] loss = 0.8647886, step = 200 (18.454 sec)
I0704 00:38:13.864868 140182873646976 basic_session_run_hooks.py:260] lr = 0.0003988925 (18.456 sec)
I0704 00:38:30.707146 140182873646976 basic_session_run_hooks.py:692] global_step/sec: 5.93527
I0704 00:38:30.712840 140182873646976 basic_session_run_hooks.py:260] loss = 0.9283235, step = 300 (16.852 sec

Reading ../data/test.txt


I0704 00:43:20.796622 140182873646976 evaluation.py:275] Finished evaluation at 2019-07-04-00:43:20
I0704 00:43:20.798022 140182873646976 estimator.py:2039] Saving dict for global step 1719: acc = 0.7357492, global_step = 1719, loss = 0.6269219
I0704 00:43:21.811380 140182873646976 estimator.py:2099] Saving 'checkpoint_path' summary for global step 1719: ../model/esim/model.ckpt-1719
W0704 00:43:22.096621 140182873646976 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/summary/summary_iterator.py:68: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
I0704 00:43:35.903943 140182873646976 basic_session_run_hooks.py:692] global_step/sec: 1.54908
I0704 00:43:35.909255 140182873646976 basic_session_run_hooks.py:260] loss = 0.6720127, step = 1800 (64.557 sec)
I0704 00:43:35.912235 140182873646976 basic_session_

...

Reading ../data/test.txt


I0704 05:32:09.507308 140182873646976 evaluation.py:275] Finished evaluation at 2019-07-04-05:32:09
I0704 05:32:09.508908 140182873646976 estimator.py:2039] Saving dict for global step 89388: acc = 0.87774837, global_step = 89388, loss = 0.3375693
I0704 05:32:09.516625 140182873646976 estimator.py:2099] Saving 'checkpoint_path' summary for global step 89388: ../model/esim/model.ckpt-89388
I0704 05:32:09.853110 140182873646976 early_stopping.py:415] No increase in metric "acc" for 12033 steps, which is greater than or equal to max steps (12033) configured for early stopping.
I0704 05:32:09.854317 140182873646976 early_stopping.py:485] Requesting early stopping at global step 89388
I0704 05:32:09.991340 140182873646976 basic_session_run_hooks.py:606] Saving checkpoints for 89389 into ../model/esim/model.ckpt.
I0704 05:32:14.449627 140182873646976 training.py:527] Skip the current checkpoint eval due to throttle secs (60 secs).
I0704 05:32:14.496976 140182873646976 estimator.py:1145] Call

Reading ../data/test.txt


I0704 05:32:56.907736 140182873646976 evaluation.py:275] Finished evaluation at 2019-07-04-05:32:56
I0704 05:32:56.909390 140182873646976 estimator.py:2039] Saving dict for global step 89389: acc = 0.8773412, global_step = 89389, loss = 0.33767843
I0704 05:32:56.916251 140182873646976 estimator.py:2099] Saving 'checkpoint_path' summary for global step 89389: ../model/esim/model.ckpt-89389
I0704 05:32:57.962899 140182873646976 estimator.py:368] Loss for final step: 0.5541159.


({'acc': 0.8773412, 'global_step': 89389, 'loss': 0.33767843}, [])